# Loading libraries and data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import roc_auc_score

In [2]:
X_train = pd.read_csv('../input/product-failure/X_train_dummies_scaled_knn.csv', index_col=0)
X_test = pd.read_csv('../input/product-failure/X_test_dummies_scaled_knn.csv', index_col=0)
y_train = pd.read_csv('../input/product-failure/y_train.csv', index_col=0)
y_test = pd.read_csv('../input/product-failure/y_test.csv', index_col=0)

Check first 5 observations and data shapes to make sure that the data was read correctly

In [3]:
X_train.head()

,loading,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,...,"measurement_2_grouped_(-0.001, 2.0]","measurement_2_grouped_(2.0, 3.0]","measurement_2_grouped_(3.0, 4.0]","measurement_2_grouped_(4.0, 5.0]","measurement_2_grouped_(5.0, 6.0]","measurement_2_grouped_(6.0, 7.0]","measurement_2_grouped_(7.0, 8.0]","measurement_2_grouped_(8.0, 9.0]","measurement_2_grouped_(9.0, 11.0]","measurement_2_grouped_(11.0, 24.0]"
0,-0.513603,2.078416,-1.485328,-0.681222,-0.653551,-0.015593,-0.319033,-0.014153,-0.067520,0.327396,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.036400,-1.070214,0.179974,-0.379065,-0.549359,-0.731627,1.703556,0.598172,-0.121901,-0.066527,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.019484,-0.828012,1.845276,-0.379065,-1.697273,0.577577,-1.657378,-0.286633,1.452140,0.304632,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.060479,0.867404,0.179974,-0.076908,-1.114317,0.677278,0.361192,-0.752161,-0.738225,0.907394,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.549740,-0.585809,-0.533727,-0.076908,-0.828338,0.043825,-1.348916,0.015005,0.087568,-1.010756,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
print("X_train.shape: ", X_train.shape)
print("y_train.shape: ", y_train.shape)
print("X_test.shape: ", X_test.shape)
print("y_test.shape:", y_test.shape)

X_train.shape:  (19927, 64)
y_train.shape:  (19927, 1)
X_test.shape:  (6643, 64)
y_test.shape: (6643, 1)


In [5]:
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

print("y_train.shape: ", y_train.shape)
print("y_test.shape:", y_test.shape)

y_train.shape:  (19927,)
y_test.shape: (6643,)


# Baseline regression model <br>
Build a baseline logistic regression model and evaluate its performance on train and test sets. For performance evaluation AUC will be used

In [6]:
logreg_baseline = LogisticRegression(solver='liblinear')
logreg_baseline.fit(X_train, y_train)

print("Train set AUC: {}".format(roc_auc_score(y_train, logreg_baseline.predict(X_train))))
print("Test set AUC: {}".format(roc_auc_score(y_test, logreg_baseline.predict(X_test))))

Train set AUC: 0.5002201080421848
Test set AUC: 0.5014588271647604


AUC values indicate, that the model may underfit the dataset. We can try to tune its parameters to improve quality of the model

# Tuning regression parameters values <br>
To tune parameters values GridSearchCV will be used. To prevent data leakage, we'll apply data transformation on each training folder separately. In order to do that, we need to read the datasets created as a result of one-hot encoding of the categorical features, but without other preprocessing steps

In [7]:
X_train_dummies = pd.read_csv('../input/product-failure/X_train_dummies.csv', index_col=0)
X_test_dummies = pd.read_csv('../input/product-failure/X_test_dummies.csv', index_col=0)

X_train_dummies = X_train_dummies.astype(np.float32)
X_test_dummies = X_test_dummies.astype(np.float32)

In [8]:
pipe = Pipeline([
    ('transformer', ColumnTransformer([
        ('scaler', StandardScaler(), [*range(0, 19)])],
         remainder='passthrough')),
    ('imputer', KNNImputer()),
    ('logreg', LogisticRegression(random_state=0, 
                                  max_iter=1000))
])

Round 1

In [9]:
param_grid = [{
    'logreg__penalty': ['elasticnet'],
    'logreg__l1_ratio': [0, 0.25, 0.5, 0.75, 1],
    'logreg__C': [0.7, 0.8, 0.9, 1, 1.1],
    'logreg__solver': ['saga']
    },
    {'logreg__penalty': ['none'],
     'logreg__solver': ['lbfgs']}
]

grid = GridSearchCV(pipe, param_grid, cv=5,
                    scoring='roc_auc',
                    verbose=2)

grid.fit(X_train_dummies, y_train)

print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Fitting 5 folds for each of 26 candidates, totalling 130 fits
[CV] END logreg__C=0.7, logreg__l1_ratio=0, logreg__penalty=elasticnet, logreg__solver=saga; total time= 1.3min
[CV] END logreg__C=0.7, logreg__l1_ratio=0, logreg__penalty=elasticnet, logreg__solver=saga; total time= 1.3min
[CV] END logreg__C=0.7, logreg__l1_ratio=0, logreg__penalty=elasticnet, logreg__solver=saga; total time=  58.3s
[CV] END logreg__C=0.7, logreg__l1_ratio=0, logreg__penalty=elasticnet, logreg__solver=saga; total time=  56.6s
[CV] END logreg__C=0.7, logreg__l1_ratio=0, logreg__penalty=elasticnet, logreg__solver=saga; total time=  56.1s
[CV] END logreg__C=0.7, logreg__l1_ratio=0.25, logreg__penalty=elasticnet, logreg__solver=saga; total time= 1.3min
[CV] END logreg__C=0.7, logreg__l1_ratio=0.25, logreg__penalty=elasticnet, logreg__solver=saga; total time= 1.3min
[CV] END logreg__C=0.7, logreg__l1_ratio=0.25, logreg__penalty=elasticnet, logreg__solver=saga; total time=  55.8s
[CV] END logreg__C=0.7, logreg__l

The result of the first round of grid search shows, that the highest AUC is obtained when penalty='elasticnet' and l1_ratio=1, which means l1 regularization. The lowest value of regularization parameter C was selected, so in the second round the C values will be lowered. Also values of l1_ratio from 0.7 to 1 will be tested to make sure that l1_ratio=1 is the optimal one

Round 2

In [10]:
param_grid = {
    'logreg__penalty': ['elasticnet'],
    'logreg__l1_ratio': [0.7, 0.8, 0.9, 1],
    'logreg__C': [0.1, 0.3, 0.5, 0.7],
    'logreg__solver': ['saga']
    }

grid = GridSearchCV(pipe, param_grid, cv=5,
                    scoring='roc_auc',
                    verbose=2)

grid.fit(X_train_dummies, y_train)

print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END logreg__C=0.1, logreg__l1_ratio=0.7, logreg__penalty=elasticnet, logreg__solver=saga; total time= 1.1min
[CV] END logreg__C=0.1, logreg__l1_ratio=0.7, logreg__penalty=elasticnet, logreg__solver=saga; total time=  57.6s
[CV] END logreg__C=0.1, logreg__l1_ratio=0.7, logreg__penalty=elasticnet, logreg__solver=saga; total time=  41.5s
[CV] END logreg__C=0.1, logreg__l1_ratio=0.7, logreg__penalty=elasticnet, logreg__solver=saga; total time=  41.4s
[CV] END logreg__C=0.1, logreg__l1_ratio=0.7, logreg__penalty=elasticnet, logreg__solver=saga; total time=  40.8s
[CV] END logreg__C=0.1, logreg__l1_ratio=0.8, logreg__penalty=elasticnet, logreg__solver=saga; total time= 1.1min
[CV] END logreg__C=0.1, logreg__l1_ratio=0.8, logreg__penalty=elasticnet, logreg__solver=saga; total time=  56.9s
[CV] END logreg__C=0.1, logreg__l1_ratio=0.8, logreg__penalty=elasticnet, logreg__solver=saga; total time=  48.6s
[CV] END logreg__C=0.1, log

At the second round the lowest value of regularization parameter was selected, so the third round models with values of C between 1e-6 and 1e-1 will be evaluated

Round 3

In [11]:
param_grid = {
    'logreg__penalty': ['elasticnet'],
    'logreg__l1_ratio': [0.9, 1],
    'logreg__C': [0.1, 0.01, 0.001, 0.0001, 1e-5, 1e-6],
    'logreg__solver': ['saga']
    }

grid = GridSearchCV(pipe, param_grid, cv=5,
                    scoring='roc_auc',
                    verbose=2)

grid.fit(X_train_dummies, y_train)

print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END logreg__C=0.1, logreg__l1_ratio=0.9, logreg__penalty=elasticnet, logreg__solver=saga; total time= 1.1min
[CV] END logreg__C=0.1, logreg__l1_ratio=0.9, logreg__penalty=elasticnet, logreg__solver=saga; total time= 1.1min
[CV] END logreg__C=0.1, logreg__l1_ratio=0.9, logreg__penalty=elasticnet, logreg__solver=saga; total time=  48.7s
[CV] END logreg__C=0.1, logreg__l1_ratio=0.9, logreg__penalty=elasticnet, logreg__solver=saga; total time=  48.1s
[CV] END logreg__C=0.1, logreg__l1_ratio=0.9, logreg__penalty=elasticnet, logreg__solver=saga; total time=  44.8s
[CV] END logreg__C=0.1, logreg__l1_ratio=1, logreg__penalty=elasticnet, logreg__solver=saga; total time=  56.7s
[CV] END logreg__C=0.1, logreg__l1_ratio=1, logreg__penalty=elasticnet, logreg__solver=saga; total time=  54.7s
[CV] END logreg__C=0.1, logreg__l1_ratio=1, logreg__penalty=elasticnet, logreg__solver=saga; total time=  30.7s
[CV] END logreg__C=0.1, logreg__l

The first 3 rounds of grid search shows, that the highest AUC is obtained using l1 regularization with C around 0.1. In the 4th round 0.15, 0.1 and 0.05 will be tested as the C parameter values

Round 4

In [12]:
param_grid = {
    'logreg__penalty': ['elasticnet'],
    'logreg__l1_ratio': [0.95, 1],
    'logreg__C': [0.15, 0.1, 0.05],
    'logreg__solver': ['saga']
    }

grid = GridSearchCV(pipe, param_grid, cv=5,
                    scoring='roc_auc',
                    verbose=2)

grid.fit(X_train_dummies, y_train)

print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END logreg__C=0.15, logreg__l1_ratio=0.95, logreg__penalty=elasticnet, logreg__solver=saga; total time=  52.8s
[CV] END logreg__C=0.15, logreg__l1_ratio=0.95, logreg__penalty=elasticnet, logreg__solver=saga; total time=  43.0s
[CV] END logreg__C=0.15, logreg__l1_ratio=0.95, logreg__penalty=elasticnet, logreg__solver=saga; total time=  41.4s
[CV] END logreg__C=0.15, logreg__l1_ratio=0.95, logreg__penalty=elasticnet, logreg__solver=saga; total time=  40.8s
[CV] END logreg__C=0.15, logreg__l1_ratio=0.95, logreg__penalty=elasticnet, logreg__solver=saga; total time=  46.8s
[CV] END logreg__C=0.15, logreg__l1_ratio=1, logreg__penalty=elasticnet, logreg__solver=saga; total time= 1.0min
[CV] END logreg__C=0.15, logreg__l1_ratio=1, logreg__penalty=elasticnet, logreg__solver=saga; total time= 1.1min
[CV] END logreg__C=0.15, logreg__l1_ratio=1, logreg__penalty=elasticnet, logreg__solver=saga; total time=  43.2s
[CV] END logreg__C=0.

Based on 4 rounds of grid search we can conclude, that for logistic regression elasticnet regularization should be used with the value of C=0.1. It gives us average AUC=0.588933

# Baseline SVM model <br>
Build a baseline (linear) SVM and evaluate its performance on train and test sets

In [13]:
svc = LinearSVC(max_iter=3000)
svc.fit(X_train, y_train)

print("Train set AUC: {}".format(roc_auc_score(y_train, logreg_baseline.predict(X_train))))
print("Test set AUC: {}".format(roc_auc_score(y_test, logreg_baseline.predict(X_test))))

Train set AUC: 0.5002201080421848
Test set AUC: 0.5014588271647604


Based on the AUC values we can conclude, that the model may underfit the dataset

# Tuning LinearSVC parameters values <br>
To tune parameters values GridSearchCV will be used. To prevent data leakage, we'll apply data transformation on each training folder separately. As before, pipelines will be used

In [14]:
pipe = Pipeline([
    ('transformer', ColumnTransformer([
        ('scaler', StandardScaler(), [*range(0, 19)])],
         remainder='passthrough')),
    ('imputer', KNNImputer()),
    ('svm', LinearSVC(max_iter=7000, dual=False))
])

In [15]:
param_grid = {
    'svm__penalty': ['l1', 'l2'],
    'svm__C': [0.001, 0.01, 0.1, 0.2, 0.3]
}

grid = GridSearchCV(pipe, param_grid, cv=5,
                    scoring='roc_auc',
                    verbose=2)

grid.fit(X_train_dummies, y_train)

print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ......................svm__C=0.001, svm__penalty=l1; total time=  59.2s
[CV] END ......................svm__C=0.001, svm__penalty=l1; total time=  49.2s
[CV] END ......................svm__C=0.001, svm__penalty=l1; total time=  32.3s
[CV] END ......................svm__C=0.001, svm__penalty=l1; total time=  38.1s
[CV] END ......................svm__C=0.001, svm__penalty=l1; total time=  41.7s
[CV] END ......................svm__C=0.001, svm__penalty=l2; total time= 1.1min
[CV] END ......................svm__C=0.001, svm__penalty=l2; total time=  46.6s
[CV] END ......................svm__C=0.001, svm__penalty=l2; total time=  30.4s
[CV] END ......................svm__C=0.001, svm__penalty=l2; total time=  34.4s
[CV] END ......................svm__C=0.001, svm__penalty=l2; total time=  28.8s
[CV] END .......................svm__C=0.01, svm__penalty=l1; total time=  45.8s
[CV] END .......................svm__C=0.01, svm

The optimal parameters of the LinearSVC model are C=0.01, penalty='l1'. It gives average AUC=0.58878959

# Tuning parameters values of SVC with non-linear kernels <br>
To obtain better results, polynomial, sigmoid and RBF kernels will be tested. To make parameters tuning process more convinient, it will be done in 2 steps. At the first step, sigmoid and RBF kernels will be tested. SVCs with polinomial kernels will be tested at the second step. <br>
<br>
## Step 1: sigmoid and RBF kernels

Round 1

In [9]:
pipe = Pipeline([
    ('transformer', ColumnTransformer([
        ('scaler', StandardScaler(), [*range(0, 19)])],
         remainder='passthrough')),
    ('imputer', KNNImputer()),
    ('svm', SVC(max_iter=15000, random_state=42))
])

In [ ]:
param_grid = [
    {'svm__kernel': ['sigmoid'],
     'svm__coef0': [0, 1],
     'svm__gamma': ['scale', 'auto'],
     'svm__C': [0.001, 0.01, 0.1, 0.3]},
    {'svm__kernel': ['rbf'],
     'svm__gamma': ['scale', 'auto'],
     'svm__C': [0.001, 0.01, 0.1, 0.3]}
]

grid = GridSearchCV(pipe, param_grid, cv=5,
                    scoring='roc_auc',
                    verbose=0)

grid.fit(X_train_dummies, y_train)

In [19]:
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Best score:  0.5353663101423228
Best params:  {'svm__C': 0.001, 'svm__coef0': 0, 'svm__gamma': 'auto', 'svm__kernel': 'sigmoid'}


As a result of the first round a sigmoid kernel was selected as the best one with C=0.001. C=0.001 is the lowest tested value of the regularization parameter, so in the second round values of C below 0.001 will be tested.

Round 2

In [13]:
pipe = Pipeline([
    ('transformer', ColumnTransformer([
        ('scaler', StandardScaler(), [*range(0, 19)])],
         remainder='passthrough')),
    ('imputer', KNNImputer()),
    ('svm', SVC(max_iter=20000, random_state=42))
])

In [17]:
param_grid = {
    'svm__kernel': ['sigmoid'],
    'svm__coef0': [0],
    'svm__gamma': ['auto'],
    'svm__C': [0.001, 0.0001, 1e-5, 1e-6]
}

grid = GridSearchCV(pipe, param_grid, cv=5,
                    scoring='roc_auc',
                    verbose=2)

grid.fit(X_train_dummies, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END svm__C=0.001, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  55.0s
[CV] END svm__C=0.001, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  46.1s
[CV] END svm__C=0.001, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  38.5s
[CV] END svm__C=0.001, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  38.8s
[CV] END svm__C=0.001, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  38.1s
[CV] END svm__C=0.0001, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  43.2s
[CV] END svm__C=0.0001, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  41.6s
[CV] END svm__C=0.0001, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  39.2s
[CV] END svm__C=0.0001, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  43.8s
[CV] END svm__C=0.0001, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  4

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scaler',
                                                                         StandardScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8, 9,
                                                                          10,
                                                                          11,
                                                                          12,
                                   

In [18]:
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Best score:  0.5425866322851085
Best params:  {'svm__C': 1e-06, 'svm__coef0': 0, 'svm__gamma': 'auto', 'svm__kernel': 'sigmoid'}


In the second round the lowest value of C was selected. So in the third round the C will be decreased.

In [19]:
param_grid = {
    'svm__kernel': ['sigmoid'],
    'svm__coef0': [0],
    'svm__gamma': ['auto'],
    'svm__C': [1e-6, 1e-7, 1e-8, 1e-9, 1e-10]
}

grid = GridSearchCV(pipe, param_grid, cv=5,
                    scoring='roc_auc',
                    verbose=2)

grid.fit(X_train_dummies, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END svm__C=1e-06, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  44.9s
[CV] END svm__C=1e-06, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  46.3s
[CV] END svm__C=1e-06, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  40.2s
[CV] END svm__C=1e-06, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  38.1s
[CV] END svm__C=1e-06, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  38.8s
[CV] END svm__C=1e-07, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  41.8s
[CV] END svm__C=1e-07, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  44.0s
[CV] END svm__C=1e-07, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  38.3s
[CV] END svm__C=1e-07, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  37.4s
[CV] END svm__C=1e-07, svm__coef0=0, svm__gamma=auto, svm__kernel=sigmoid; total time=  37.6s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scaler',
                                                                         StandardScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8, 9,
                                                                          10,
                                                                          11,
                                                                          12,
                                   

In [20]:
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Best score:  0.5425866322851085
Best params:  {'svm__C': 1e-06, 'svm__coef0': 0, 'svm__gamma': 'auto', 'svm__kernel': 'sigmoid'}


The optimal value of C is 1e-6. With C=1e-06 and sigmoid kernel, AUC=0.5425867 was achieved.

## Step 2: polynomial kernel

In [10]:
param_grid = {
    'svm__kernel': ['poly'],
    'svm__degree': [3, 2],
    'svm__coef0': [0, 1],
    'svm__gamma': ['scale', 'auto'],
    'svm__C': [0.001, 0.01, 0.1, 0.3]
}

grid = GridSearchCV(pipe, param_grid, cv=5,
                    scoring='roc_auc',
                    verbose=2)

grid.fit(X_train_dummies, y_train)
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=scale, svm__kernel=poly; total time=  40.9s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=scale, svm__kernel=poly; total time=  38.8s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=scale, svm__kernel=poly; total time=  34.6s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=scale, svm__kernel=poly; total time=  34.4s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=scale, svm__kernel=poly; total time=  35.0s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=  35.4s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=  36.9s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=  39.2s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=15000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


[CV] END svm__C=0.01, svm__coef0=0, svm__degree=3, svm__gamma=scale, svm__kernel=poly; total time= 1.4min


/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=15000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


[CV] END svm__C=0.01, svm__coef0=0, svm__degree=3, svm__gamma=scale, svm__kernel=poly; total time= 1.0min


...

[CV] END svm__C=0.3, svm__coef0=1, svm__degree=2, svm__gamma=auto, svm__kernel=poly; total time=  37.6s


/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=15000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


[CV] END svm__C=0.3, svm__coef0=1, svm__degree=2, svm__gamma=auto, svm__kernel=poly; total time=  37.9s


/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=15000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


[CV] END svm__C=0.3, svm__coef0=1, svm__degree=2, svm__gamma=auto, svm__kernel=poly; total time=  37.7s
Best score:  0.5354049795495074
Best params:  {'svm__C': 0.001, 'svm__coef0': 0, 'svm__degree': 3, 'svm__gamma': 'auto', 'svm__kernel': 'poly'}


The optimal values for degree, coef0 and gamma parameters are respectively 3, 0 and 'auto'. Since in the first round the lowest value of C was selected, in the second round C will be decreased.

In [21]:
param_grid = {
    'svm__kernel': ['poly'],
    'svm__degree': [3],
    'svm__coef0': [0],
    'svm__gamma': ['auto'],
    'svm__C': [0.001, 0.0001, 1e-5, 1e-6, 1e-7]
}

grid = GridSearchCV(pipe, param_grid, cv=5,
                    scoring='roc_auc',
                    verbose=2)

grid.fit(X_train_dummies, y_train)
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=  37.2s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=  37.1s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=  34.7s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=  33.9s
[CV] END svm__C=0.001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=  33.3s
[CV] END svm__C=0.0001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=  36.9s
[CV] END svm__C=0.0001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=  37.5s
[CV] END svm__C=0.0001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; total time=  33.9s
[CV] END svm__C=0.0001, svm__coef0=0, svm__degree=3, svm__gamma=auto, svm__kernel=poly; t

In [22]:
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

Best score:  0.536925344721615
Best params:  {'svm__C': 0.0001, 'svm__coef0': 0, 'svm__degree': 3, 'svm__gamma': 'auto', 'svm__kernel': 'poly'}


The optimal value for C parameter is 0.0001 with coef0=0, degree=3 and gamma='auto'.

## SVC tuning summary

In [23]:
## SVC tuning summary